In [ ]:
import numpy as np
import hyperspy.api as hys
import matplotlib.pyplot as plt
import tkinter.filedialog as tkf
import tifffile
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
si_adr = []

In [ ]:
si_adr.extend(tkf.askopenfilenames())
print(len(si_adr))
print(*si_adr, sep="\n")

In [ ]:
title = []
peak_ratio_maps = []
peak_ratio_mean = []
peak_ratio_median = []

two_window_maps = []
two_win_mean = []
two_win_median = []

two_window_2_maps = []
two_win_2_mean = []
two_win_2_median = []

num_si = len(si_adr)

for adr in si_adr:
    tmp_title = adr.split("/")[-1]
    print(tmp_title)
    title.append(tmp_title)
    si = hys.load(adr, signal_type="EELS")

    ini_channel = 280.0
    fin_channel = 315.0
    dispersion = np.around(si.axes_manager[2].scale, 4)
    e_range = np.arange(ini_channel, fin_channel, dispersion)

    si = si.isig[ini_channel:fin_channel]
    data = si.data.clip(min=0.0)
    si_shape = data.shape

    data_flat = data.reshape(-1, si_shape[2])

    pi_onset = 282.0
    pi_pos = 285.0
    pi_final = 287.0
    sigma_onset = 290.0
    sigma_pos = 293.0
    sigma_mid = 295.0
    sigma_final = 315.0

    pi_ind = int((pi_pos - e_range[0]) / dispersion)
    pi_onset_ind = int((pi_onset - e_range[0]) / dispersion)
    pi_final_ind = int((pi_final - e_range[0]) / dispersion)
    sigma_ind = int((sigma_pos - e_range[0]) / dispersion)
    sigma_onset_ind = int((sigma_onset - e_range[0]) / dispersion)
    sigma_mid_ind = int((sigma_mid - e_range[0]) / dispersion)
    sigma_final_ind = int((sigma_final - e_range[0]) / dispersion)

    pi_range = e_range[pi_onset_ind:pi_final_ind]
    pi_sigma_range = e_range[pi_onset_ind:sigma_final_ind]
    sigma_range = e_range[sigma_onset_ind:sigma_mid_ind]

    peak_ratio_map = []
    two_window_map = []
    two_window_2_map = []
    for i, s in enumerate(data_flat):
        tmp = s - np.min(s)
        pi_int = tmp[pi_ind]
        sigma_int = tmp[sigma_ind]
        if sigma_int != 0.0:
            peak_ratio_map.append(pi_int/sigma_int)
        else:
            peak_ratio_map.append(0)

        pi_spec = tmp[pi_onset_ind:pi_final_ind]
        pi_sigma_spec = tmp[pi_onset_ind:sigma_final_ind]

        Epi = np.trapz(pi_spec, pi_range)
        Episigma = np.trapz(pi_sigma_spec, pi_sigma_range)
        
        if Episigma != 0.0:
            two_window_map.append(Epi/Episigma)
        else:
            two_window_map.append(0)
            
        sigma_spec = tmp[sigma_onset_ind:sigma_mid_ind]
        Esigma = np.trapz(sigma_spec, sigma_range)
        
        if Esigma != 0.0:
            two_window_2_map.append(Epi/Esigma)
        else:
            two_window_2_map.append(0)
    
    print("peak ratio")
    peak_ratio_map = np.asarray(peak_ratio_map).reshape(si_shape[0], si_shape[1])
    print(peak_ratio_map.shape)
    print(np.min(peak_ratio_map), np.max(peak_ratio_map))
    print("two window method 1")
    two_window_map = np.asarray(two_window_map).reshape(si_shape[0], si_shape[1])
    print(two_window_map.shape)
    print(np.min(two_window_map), np.max(two_window_map))
    print("two window method 2")
    two_window_2_map = np.asarray(two_window_2_map).reshape(si_shape[0], si_shape[1])
    print(two_window_2_map.shape)
    print(np.min(two_window_2_map), np.max(two_window_2_map))
    
    peak_ratio_maps.append(peak_ratio_map)
    peak_ratio_mean.append(np.mean(peak_ratio_map))
    peak_ratio_median.append(np.median(peak_ratio_map))
    two_window_maps.append(two_window_map)
    two_win_mean.append(np.mean(two_window_map))
    two_win_median.append(np.median(two_window_map))
    two_window_2_maps.append(two_window_2_map)
    two_win_2_mean.append(np.mean(two_window_2_map))
    two_win_2_median.append(np.median(two_window_2_map))    

In [ ]:
for z in zip(title, peak_ratio_mean, peak_ratio_median):
    print(z)

In [ ]:
for z in zip(title, two_win_mean, two_win_median):
    print(z)

In [ ]:
for z in zip(title, two_win_2_mean, two_win_2_median):
    print(z)

In [ ]:
%matplotlib widget

In [ ]:
%matplotlib inline

In [ ]:
plt.close()
for i in range(num_si):
    fig, ax = plt.subplots(1, 3, figsize=(10, 3))
    fig.suptitle(title[i])
    ax[0].imshow(peak_ratio_maps[i], cmap="inferno", vmin=0.0, vmax=0.8)
    ax[0].axis("off")
    ax[1].imshow(two_window_maps[i], cmap="inferno", vmin=0.04, vmax=0.10)
    ax[1].axis("off")
    ax[2].imshow(two_window_2_maps[i], cmap="inferno", vmin=0.2, vmax=0.5)
    ax[2].axis("off")
    fig.tight_layout()
    plt.show()